In [1]:
# Carga paquetes
import pandas as pd
import numpy as np
import warnings
import re

warnings.filterwarnings("ignore", category=UserWarning)  # ignora los warnings

In [2]:
###########    FUNCIONES   ##########
## Interquartile Range [IQR)
#Se calculan los limites superiores e inferiores para identificación de posibles datos atipicos
# IQR = Q3 - Q1 
# LI = Q1–1.5 * IQR; LS = Q3+1.5 * IQR.
def IQR(variable):
  Iqr = np.percentile(variable,75)-np.percentile(variable,25)

  LI=np.percentile(variable,25)-(1.5*Iqr)
  LS=np.percentile(variable,75)+(1.5*Iqr)
  return(LI,LS)

In [3]:
def categorizacion (variable,outliers,reciencia):
  LI = outliers[0]
  LS = outliers[1]

  # variable tiene outliers, VarTemp no
  VarTemp = variable[(variable >= LI) & (variable <= LS)]

  ## TIPO DE INTERVALO O BREAK  
  # valores en el que se dividen los intervalos. 
  # los datos atipicos se apartan a la hora de definir los quantiles, pero se incluyen al final para el puntaje
  # break_1: <LI + min; percentiles -1 y 6; max + >LS
  breaks_1 = np.concatenate(([min(variable) - 0.001]\
    ,np.percentile(VarTemp, np.arange(0, 101, 20)[1:5]) #np.arange(start, stop, step), stop excluded so +1/
    ,[max(variable) + 0.001]))

  # break_2: quantiles con outliers
  quantiles_2 = np.percentile(variable, np.arange(0, 101, 20))
  quantiles_2[0] -= 0.001
  quantiles_2[-1] += 0.001
  breaks_2 = quantiles_2
 
  # break_3: Usandos rangos fijos y no quantiles incluyendo outliers
  breaks_3 = np.linspace(min(variable) - 0.001, max(variable) + 0.001, num=6)

  if not reciencia:
    # Usar el break sin outliers sin que quantiles repitan valores
    if not np.any(pd.Series(breaks_1).duplicated()):
        puntaje = pd.cut(variable, bins=breaks_1, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_1)
    # Usar el break con outliers sin que quantiles repitan valores
    elif np.any(pd.Series(breaks_1).duplicated()) and not np.any(pd.Series(breaks_2).duplicated()):
        puntaje = pd.cut(variable, bins=breaks_2, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_2)
    # Usar el break con outliers sin usar quantiles sino rangos fijos    
    else:
        puntaje = pd.cut(variable, bins=breaks_3, labels=np.arange(1, 6)).astype(int)
        categorias = pd.cut(variable, bins=breaks_3)

# restar 6 y multiplicar -1 invierte el puntaje para Recencia, es decir, si el puntaje mayor es 5 para los valores mas grandes 
# en los otros casos, en Recencia los menores son los de mayor puntaje
  else:
    if not np.any(pd.Series(breaks_1).duplicated()):
        puntaje = ((pd.cut(variable, bins=breaks_1, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_1)
    elif np.any(pd.Series(breaks_1).duplicated()) and not np.any(pd.Series(breaks_2).duplicated()):
        puntaje = ((pd.cut(variable, bins=breaks_2, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_2)
    else:
        puntaje = ((pd.cut(variable, bins=breaks_3, labels=np.arange(1, 6)).astype(int)) - 6) * (-1)
        categorias = pd.cut(variable, bins=breaks_3)

  return (puntaje,categorias)

In [4]:
#Función para el caso WHEN
def case_when(x):
    if x in[17,18,19,20]:
        return 'Muy Alto'
    elif x in[13,14,15,16]:
        return 'Alto'
    elif x in[8,9,10,11,12]:
        return 'Medio'
    else:
        return 'Bajo'

In [5]:
def puntajes_RFM (tb):
  #Ejecutamos las dos funciones anteriores
  rfm_df = tb
  lim_rec = IQR(rfm_df['Recencia'])
  lim_freq = IQR(rfm_df['Frecuencia'])
  lim_monto = IQR(rfm_df['Monto'])

  cat_rec = categorizacion(rfm_df['Recencia'], lim_rec, reciencia= True)
  cat_freq = categorizacion(rfm_df['Frecuencia'], lim_freq, reciencia=False)
  cat_monto = categorizacion(rfm_df['Monto'], lim_monto, reciencia=False)

  # Campos de puntaje
  rfm_df['puntaje_recencia'] = cat_rec[0]
  rfm_df['puntaje_frecuencia'] = cat_freq[0]
  rfm_df['puntaje_monto'] = cat_monto[0]

  # Campos de rango 
  rfm_df['rango_recencia'] = cat_rec[1]
  rfm_df['rango_frecuencia'] = cat_freq[1]
  rfm_df['rango_monto'] = cat_monto[1]
  
  rfm_df['score'] = rfm_df['puntaje_recencia'].astype(str)\
     + rfm_df['puntaje_frecuencia'].astype(str)\
     + rfm_df['puntaje_monto'].astype(str)
  
  rfm_df['score'] = rfm_df['score'].astype(int)

  #rfm_df['Score'] = rfm_df['puntaje_recencia'] + rfm_df['puntaje_frecuencia'] + rfm_df['puntaje_monto']
  
  rfm_df['Valor'] = rfm_df['score'].apply(lambda x: case_when(x))

  return(rfm_df)


In [8]:
# Carga de la base
#inpath = "C:/Users/Diego Torres/OneDrive/Datasets/Manar/"
inpath = "C:/Users/diego.torres/OneDrive/Datasets/Manar/"

tb_ini = pd.read_excel(inpath + 'RFM_pruebas.xlsx',
                       sheet_name="Hoja1")

In [9]:
#################
# Descriptivos
print(tb_ini.sample(5))
print(' ')
print('Summary:')
tb_ini.info()
print(' ')
print('Descriptivos:')
print(tb_ini.describe().round(2).transpose())
print(' ')
print('Nulos por campo:')
print(tb_ini.isnull().sum())  # total de nulls por variable
print(' ')
print('Dimensiones:')
print(tb_ini.shape)

        codigo cliente  nombre cliente  codigo producto  \
67738        113207675       113207675         31000705   
91724     133110641989    133110641989         32408002   
2369         113249240       113249240         31000705   
68382        112289941       112289941         32104001   
102937     13350006439     13350006439         32408002   

                                          nombre producto fecha venta  \
67738   MT-IBC MENSUALIDAD FIJA ADMINISTRACION CEN-TEN...  2022-06-02   
91724               DIF MT-IBC MENSUALIDAD CEN FINANCIERO  2022-07-25   
2369    MT-IBC MENSUALIDAD FIJA ADMINISTRACION CEN-TEN...  2022-01-07   
68382        MT-IBC MENSUALIDAD NUMERO TRANSACCIONES CENT  2022-06-02   
102937              DIF MT-IBC MENSUALIDAD CEN FINANCIERO  2022-08-25   

        cantidad facturado  valor facturado               Tipo  
67738                    1          80459.0  CEN TRANSACCIONAL  
91724                    1          46171.0     CEN FINANCIERO  
2369       

In [10]:
####   RFM CLIENTES

# esta linea genera la fecha a la que comparar
hoy = max(tb_ini['fecha venta'])

tb_grp = tb_ini\
    .groupby('codigo cliente')\
    .agg(
        ultimo_dia=('fecha venta', 'max'),
        Frecuencia=('cantidad facturado', 'sum'),
        Monto=('valor facturado', 'sum'),
        nombre_cliente=('nombre cliente', 'unique'))\
    .assign(Recencia=lambda x: (hoy - x['ultimo_dia']).dt.days)\
    .sort_values('codigo cliente')
    #.assign(Recencia=(tb['recencia'].apply(lambda y: y.days)))

print(tb_grp.sample(5))
print(' ')
print('Dimensiones:')
print(tb_grp.shape)
print(' ')
print('Summary:')
tb_grp.info()

               ultimo_dia  Frecuencia      Monto  nombre_cliente  Recencia
codigo cliente                                                            
113273670      2022-08-02          14  2099872.0     [113273670]        29
113267896      2022-08-02           8   920437.0     [113267896]        29
112951895      2022-08-02          16  1489119.0     [112951895]        29
113395996      2022-08-02           3  1785000.0     [113395996]        29
133110487999   2022-08-25          56  4616702.0  [133110487999]         6
 
Dimensiones:
(3822, 5)
 
Summary:
<class 'pandas.core.frame.DataFrame'>
Index: 3822 entries, 1141132 to 139110576375
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ultimo_dia      3822 non-null   datetime64[ns]
 1   Frecuencia      3822 non-null   int64         
 2   Monto           3822 non-null   float64       
 3   nombre_cliente  3822 non-null   object        
 4   Recenci

In [ ]:
#############################
### TABLA RESULTADO
base_resultado = puntajes_RFM(tb_grp)

print(base_resultado.sample(5))
print(' ')
print('Dimensiones:')
print(base_resultado.shape)
print(' ')
print('Summary:')
base_resultado.info()